# Tabular Januari 2022

Made by: Farel Arden

Hey Happy New Year!

Here, we will predict the sales of Kaggle Store in the future.

Hopefully this will be a reality, I really want a Kaggle Sweater for my own. It looks dope!

<img src= "https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F59561%2Fcc6d9a21f0c3ed71b00113b33efb2b66%2Fkaggle_sweater.png?generation=1640900016906235&alt=media" alt ="Kaggle Store" style='width: 400px;'>

## Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import gc
from fbprophet import Prophet
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve,RocCurveDisplay,ConfusionMatrixDisplay,confusion_matrix,roc_auc_score,accuracy_score
from catboost import CatBoostClassifier
from sklearn.ensemble import (AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier,GradientBoostingClassifier,RandomForestClassifier,VotingClassifier)
from IPython.display import clear_output

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os, psutil
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Installing Prophet Model
Let's install the prophet model

In [ ]:
# !pip install pystan
# !pip install fbprophet

## Importing the Datasets

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv")

train_copy = train

Here is the function to reduce the memory and time to process the dataset

In [ ]:
%%time
# taken from https://www.kaggle.com/ryanholbrook/getting-started-september-2021-tabular-playground

def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))

def score(X, y, model, cv):
    scoring = ["roc_auc"]
    scores = cross_validate(
        model, X_train, y_train, scoring=scoring, cv=cv, return_train_score=True
    )
    scores = pd.DataFrame(scores).T
    return scores.assign(
        mean = lambda x: x.mean(axis=1),
        std = lambda x: x.std(axis=1),
    )

## from: https://www.kaggle.com/bextuychiev/how-to-work-w-million-row-datasets-like-a-pro
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

print('Function built')

In [ ]:
train = reduce_memory_usage(train, verbose=True)
test = reduce_memory_usage(test, verbose=True)
print(cpu_stats())
print('Memory reduced')

In [ ]:
train_copy.head()

Here we will analyze the unique values of the dataset

In [ ]:
countries = train_copy['country'].unique()
stores = train_copy['store'].unique()
products = train_copy['product'].unique()

print("The countries that sold the products are {}".format(countries))
print("The stores that sold the products are {}".format(stores))
print("The products that are sold are {}".format(products))

## Dividing the Datasets

Here, I am planning to divide the datasets based on its country, store, and product.

We will train the datasets individually, so bear with me, there are 18 datasets.

### Finland

In [ ]:
df_Finland_KaggleMart = train_copy[(train_copy['country'] == "Finland") & (train_copy['store'] == "KaggleMart")]
df_Finland_KaggleMart.head()

In [ ]:
df_Finland_KaggleMart_KaggleMug = df_Finland_KaggleMart[df_Finland_KaggleMart['product'] == "Kaggle Mug"]
df_Finland_KaggleMart_KaggleHat = df_Finland_KaggleMart[df_Finland_KaggleMart['product'] == "Kaggle Hat"]
df_Finland_KaggleMart_KaggleSticker = df_Finland_KaggleMart[df_Finland_KaggleMart['product'] == "Kaggle Sticker"]

In [ ]:
df_Finland_KaggleRama = train_copy[(train_copy['country'] == "Finland") & (train_copy['store'] == "KaggleRama")]
df_Finland_KaggleRama.head()

In [ ]:
df_Finland_KaggleRama_KaggleMug = df_Finland_KaggleMart[df_Finland_KaggleMart['product'] == "Kaggle Mug"]
df_Finland_KaggleRama_KaggleHat = df_Finland_KaggleMart[df_Finland_KaggleMart['product'] == "Kaggle Hat"]
df_Finland_KaggleRama_KaggleSticker = df_Finland_KaggleMart[df_Finland_KaggleMart['product'] == "Kaggle Sticker"]

### Norway

In [ ]:
df_Norway_KaggleMart = train_copy[(train_copy['country'] == "Norway") & (train_copy['store'] == "KaggleMart")]
df_Norway_KaggleMart.head()

In [ ]:
df_Norway_KaggleMart_KaggleMug = df_Norway_KaggleMart[df_Norway_KaggleMart['product'] == "Kaggle Mug"]
df_Norway_KaggleMart_KaggleHat = df_Norway_KaggleMart[df_Norway_KaggleMart['product'] == "Kaggle Hat"]
df_Norway_KaggleMart_KaggleSticker = df_Norway_KaggleMart[df_Norway_KaggleMart['product'] == "Kaggle Sticker"]

In [ ]:
df_Norway_KaggleRama = train_copy[(train_copy['country'] == "Norway") & (train_copy['store'] == "KaggleRama")]
df_Norway_KaggleRama.head()

In [ ]:
df_Norway_KaggleRama_KaggleMug = df_Norway_KaggleRama[df_Norway_KaggleRama['product'] == "Kaggle Mug"]
df_Norway_KaggleRama_KaggleHat = df_Norway_KaggleRama[df_Norway_KaggleRama['product'] == "Kaggle Hat"]
df_Norway_KaggleRama_KaggleSticker = df_Norway_KaggleRama[df_Norway_KaggleRama['product'] == "Kaggle Sticker"]

### Sweden

In [ ]:
df_Sweden_KaggleMart = train_copy[(train_copy['country'] == "Sweden") & (train_copy['store'] == "KaggleMart")]
df_Sweden_KaggleMart.head()

In [ ]:
df_Sweden_KaggleMart_KaggleMug = df_Sweden_KaggleMart[df_Sweden_KaggleMart['product'] == "Kaggle Mug"]
df_Sweden_KaggleMart_KaggleHat = df_Sweden_KaggleMart[df_Sweden_KaggleMart['product'] == "Kaggle Hat"]
df_Sweden_KaggleMart_KaggleSticker = df_Sweden_KaggleMart[df_Sweden_KaggleMart['product'] == "Kaggle Sticker"]

In [ ]:
df_Sweden_KaggleRama = train_copy[(train_copy['country'] == "Sweden") & (train_copy['store'] == "KaggleRama")]
df_Sweden_KaggleRama.head()

In [ ]:
df_Sweden_KaggleRama_KaggleMug = df_Sweden_KaggleRama[df_Sweden_KaggleRama['product'] == "Kaggle Mug"]
df_Sweden_KaggleRama_KaggleHat = df_Sweden_KaggleRama[df_Sweden_KaggleRama['product'] == "Kaggle Hat"]
df_Sweden_KaggleRama_KaggleSticker = df_Sweden_KaggleRama[df_Sweden_KaggleRama['product'] == "Kaggle Sticker"]

___

## Data Preprocessing

In [ ]:
df_Finland_KaggleMart_KaggleMug

In [ ]:
df_Finland_KaggleMart_KaggleMug_preprocessed = df_Finland_KaggleMart_KaggleMug[['date','num_sold']]
df_Finland_KaggleMart_KaggleMug_preprocessed.head()

In [ ]:
df_Finland_KaggleMart_KaggleMug_preprocessed.date = pd.to_datetime(df_Finland_KaggleMart_KaggleMug_preprocessed.date)
df_Finland_KaggleMart_KaggleMug_preprocessed.columns = ['ds', 'y']

In [ ]:
df_Finland_KaggleMart_KaggleMug_preprocessed.dtypes

___
## Data Modeling

Now we're headed to the fun part, the part where we train the datasets

In [ ]:
m = Prophet(interval_width=0.95, daily_seasonality=True)
model = m.fit(df_Finland_KaggleMart_KaggleMug_preprocessed)

In [ ]:
# Here we predict 1000 days to the future
future = m.make_future_dataframe(periods=1000,freq='D')

Okay, let me immerse you to the facts. 

Prophet doesn't only tell you what is the predicted value, but the model also add other parameters, such as trend, the boundaries, etc.

Used it as your advantage :D

In [ ]:
forecast = m.predict(future)
forecast.head()

In [ ]:
forecast.tail()

In [ ]:
plot1 = m.plot(forecast)

In [ ]:
plt2 = m.plot_components(forecast)

I am masking the dataset, because we only need 1 year of 2019 to be submitted

In [ ]:
mask = (forecast['ds'] > '2018-12-31')& (forecast['ds'] <'2020-01-01')
forecast_mask = forecast.loc[mask]

In [ ]:
forecast_mask.tail()

___
## Functioning

Because we have 18 datasets,
we conclude the cells that we have and inserted them all in a function

In [ ]:
#function
def predict(dataset):
    df_preprocessed = dataset[['date','num_sold']]
    df_preprocessed.date = pd.to_datetime(df_preprocessed.date)
    df_preprocessed.columns = ['ds', 'y']
    m = Prophet(interval_width=0.95, daily_seasonality=True)
    model = m.fit(df_preprocessed)
    future = m.make_future_dataframe(periods=1000,freq='D')
    forecast = m.predict(future)
    mask = (forecast['ds'] > '2018-12-31')& (forecast['ds'] <'2020-01-01')
    forecast_mask = forecast.loc[mask]
    forecast_mask['ds'] = forecast_mask['ds'].dt.strftime('%Y-%m-%d')
    return forecast_mask

In [ ]:
forecast_Finland_KaggleMart_KaggleMug = predict(df_Finland_KaggleMart_KaggleMug)
forecast_Finland_KaggleMart_KaggleHat = predict(df_Finland_KaggleMart_KaggleHat)
forecast_Finland_KaggleMart_KaggleSticker = predict(df_Finland_KaggleMart_KaggleSticker)
forecast_Finland_KaggleRama_KaggleMug = predict(df_Finland_KaggleRama_KaggleMug)
forecast_Finland_KaggleRama_KaggleHat = predict(df_Finland_KaggleRama_KaggleHat)
forecast_Finland_KaggleRama_KaggleSticker = predict(df_Finland_KaggleRama_KaggleSticker)
forecast_Norway_KaggleMart_KaggleMug = predict(df_Norway_KaggleMart_KaggleMug) 
forecast_Norway_KaggleMart_KaggleHat = predict(df_Norway_KaggleMart_KaggleHat) 
forecast_Norway_KaggleMart_KaggleSticker = predict(df_Norway_KaggleMart_KaggleSticker) 
forecast_Norway_KaggleRama_KaggleMug = predict(df_Norway_KaggleRama_KaggleMug) 
forecast_Norway_KaggleRama_KaggleHat = predict(df_Norway_KaggleRama_KaggleHat) 
forecast_Norway_KaggleRama_KaggleSticker = predict(df_Norway_KaggleRama_KaggleSticker) 
forecast_Sweden_KaggleMart_KaggleMug = predict(df_Sweden_KaggleMart_KaggleMug) 
forecast_Sweden_KaggleMart_KaggleHat = predict(df_Sweden_KaggleMart_KaggleHat) 
forecast_Sweden_KaggleMart_KaggleSticker = predict(df_Sweden_KaggleMart_KaggleSticker) 
forecast_Sweden_KaggleRama_KaggleMug = predict(df_Sweden_KaggleRama_KaggleMug)
forecast_Sweden_KaggleRama_KaggleHat = predict(df_Sweden_KaggleRama_KaggleHat)
forecast_Sweden_KaggleRama_KaggleSticker = predict(df_Sweden_KaggleRama_KaggleSticker)

The function below is used to preprocessed the forecasted dataframes before merged with the test data

In [ ]:
def forecast_processing(forecast):
    forecast.drop(forecast.columns.difference(['ds','yhat']), 1, inplace=True)
    forecast = forecast.rename(columns={'ds': 'date', 'yhat': 'num_sold'})
#     forecast.num_sold = forecast.num_sold.astype(int)
    return forecast

In [ ]:
forecast_Finland_KaggleMart_KaggleMug = forecast_processing(forecast_Finland_KaggleMart_KaggleMug)
forecast_Finland_KaggleMart_KaggleHat = forecast_processing(forecast_Finland_KaggleMart_KaggleHat)
forecast_Finland_KaggleMart_KaggleSticker = forecast_processing(forecast_Finland_KaggleMart_KaggleSticker)
forecast_Finland_KaggleRama_KaggleMug = forecast_processing(forecast_Finland_KaggleRama_KaggleMug)
forecast_Finland_KaggleRama_KaggleHat = forecast_processing(forecast_Finland_KaggleRama_KaggleHat)
forecast_Finland_KaggleRama_KaggleSticker = forecast_processing(forecast_Finland_KaggleRama_KaggleSticker)
forecast_Norway_KaggleMart_KaggleMug = forecast_processing(forecast_Norway_KaggleMart_KaggleMug) 
forecast_Norway_KaggleMart_KaggleHat = forecast_processing(forecast_Norway_KaggleMart_KaggleHat) 
forecast_Norway_KaggleMart_KaggleSticker = forecast_processing(forecast_Norway_KaggleMart_KaggleSticker) 
forecast_Norway_KaggleRama_KaggleMug = forecast_processing(forecast_Norway_KaggleRama_KaggleMug) 
forecast_Norway_KaggleRama_KaggleHat = forecast_processing(forecast_Norway_KaggleRama_KaggleHat) 
forecast_Norway_KaggleRama_KaggleSticker = forecast_processing(forecast_Norway_KaggleRama_KaggleSticker) 
forecast_Sweden_KaggleMart_KaggleMug = forecast_processing(forecast_Sweden_KaggleMart_KaggleMug) 
forecast_Sweden_KaggleMart_KaggleHat = forecast_processing(forecast_Sweden_KaggleMart_KaggleHat) 
forecast_Sweden_KaggleMart_KaggleSticker = forecast_processing(forecast_Sweden_KaggleMart_KaggleSticker) 
forecast_Sweden_KaggleRama_KaggleMug = forecast_processing(forecast_Sweden_KaggleRama_KaggleMug)
forecast_Sweden_KaggleRama_KaggleHat = forecast_processing(forecast_Sweden_KaggleRama_KaggleHat)
forecast_Sweden_KaggleRama_KaggleSticker = forecast_processing(forecast_Sweden_KaggleRama_KaggleSticker)

## Test Merging

In [ ]:
test_copy = test

Same as the train, we need to seperate the test dataframe based on country, store, and product.

In [ ]:
test_Finland_KaggleMart = test_copy[(test_copy['country'] == "Finland") & (test_copy['store'] == "KaggleMart")]
test_Finland_KaggleMart_KaggleMug = test_Finland_KaggleMart[test_Finland_KaggleMart['product'] == "Kaggle Mug"]
test_Finland_KaggleMart_KaggleHat = test_Finland_KaggleMart[test_Finland_KaggleMart['product'] == "Kaggle Hat"]
test_Finland_KaggleMart_KaggleSticker = test_Finland_KaggleMart[test_Finland_KaggleMart['product'] == "Kaggle Sticker"]

test_Finland_KaggleRama = test_copy[(test_copy['country'] == "Finland") & (test_copy['store'] == "KaggleRama")]
test_Finland_KaggleRama_KaggleMug = test_Finland_KaggleRama[test_Finland_KaggleRama['product'] == "Kaggle Mug"]
test_Finland_KaggleRama_KaggleHat = test_Finland_KaggleRama[test_Finland_KaggleRama['product'] == "Kaggle Hat"]
test_Finland_KaggleRama_KaggleSticker = test_Finland_KaggleRama[test_Finland_KaggleRama['product'] == "Kaggle Sticker"]

test_Norway_KaggleMart = test_copy[(test_copy['country'] == "Norway") & (test_copy['store'] == "KaggleMart")]
test_Norway_KaggleMart_KaggleMug = test_Norway_KaggleMart[test_Norway_KaggleMart['product'] == "Kaggle Mug"]
test_Norway_KaggleMart_KaggleHat = test_Norway_KaggleMart[test_Norway_KaggleMart['product'] == "Kaggle Hat"]
test_Norway_KaggleMart_KaggleSticker = test_Norway_KaggleMart[test_Norway_KaggleMart['product'] == "Kaggle Sticker"]

test_Norway_KaggleRama = test_copy[(test_copy['country'] == "Norway") & (test_copy['store'] == "KaggleRama")]
test_Norway_KaggleRama_KaggleMug = test_Norway_KaggleRama[test_Norway_KaggleRama['product'] == "Kaggle Mug"]
test_Norway_KaggleRama_KaggleHat = test_Norway_KaggleRama[test_Norway_KaggleRama['product'] == "Kaggle Hat"]
test_Norway_KaggleRama_KaggleSticker = test_Norway_KaggleRama[test_Norway_KaggleRama['product'] == "Kaggle Sticker"]

test_Sweden_KaggleMart = test_copy[(test_copy['country'] == "Sweden") & (test_copy['store'] == "KaggleMart")]
test_Sweden_KaggleMart_KaggleMug = test_Sweden_KaggleMart[test_Sweden_KaggleMart['product'] == "Kaggle Mug"]
test_Sweden_KaggleMart_KaggleHat = test_Sweden_KaggleMart[test_Sweden_KaggleMart['product'] == "Kaggle Hat"]
test_Sweden_KaggleMart_KaggleSticker = test_Sweden_KaggleMart[test_Sweden_KaggleMart['product'] == "Kaggle Sticker"]

test_Sweden_KaggleRama = test_copy[(test_copy['country'] == "Sweden") & (test_copy['store'] == "KaggleRama")]
test_Sweden_KaggleRama_KaggleMug = test_Sweden_KaggleRama[test_Sweden_KaggleRama['product'] == "Kaggle Mug"]
test_Sweden_KaggleRama_KaggleHat = test_Sweden_KaggleRama[test_Sweden_KaggleRama['product'] == "Kaggle Hat"]
test_Sweden_KaggleRama_KaggleSticker = test_Sweden_KaggleRama[test_Sweden_KaggleRama['product'] == "Kaggle Sticker"]

In [ ]:
forecast_Finland_KaggleMart_KaggleMug

In [ ]:
test_Finland_KaggleMart_KaggleMug

In [ ]:
# forecast_Finland_KaggleMart_KaggleMug['date'] = forecast_Finland_KaggleMart_KaggleMug['date'].dt.strftime('%Y-%m-%d')

In [ ]:
forecast_Finland_KaggleMart_KaggleMug

In [ ]:
forecast_Finland_KaggleMart_KaggleMug

Here we merge the seperated datasets and the forecasted number of sold items

In [ ]:
test_Finland_KaggleMart_KaggleMug = test_Finland_KaggleMart_KaggleMug.merge(forecast_Finland_KaggleMart_KaggleMug, on='date')
test_Finland_KaggleMart_KaggleHat = test_Finland_KaggleMart_KaggleHat.merge(forecast_Finland_KaggleMart_KaggleHat, on='date')
test_Finland_KaggleMart_KaggleSticker = test_Finland_KaggleMart_KaggleSticker.merge(forecast_Finland_KaggleMart_KaggleSticker, on='date')
test_Finland_KaggleRama_KaggleMug = test_Finland_KaggleRama_KaggleMug.merge(forecast_Finland_KaggleRama_KaggleMug, on='date')
test_Finland_KaggleRama_KaggleHat = test_Finland_KaggleRama_KaggleHat.merge(forecast_Finland_KaggleRama_KaggleHat, on='date')
test_Finland_KaggleRama_KaggleSticker = test_Finland_KaggleRama_KaggleSticker.merge(forecast_Finland_KaggleRama_KaggleSticker, on='date')
test_Norway_KaggleMart_KaggleMug = test_Norway_KaggleMart_KaggleMug.merge(forecast_Norway_KaggleMart_KaggleMug, on='date')
test_Norway_KaggleMart_KaggleHat = test_Norway_KaggleMart_KaggleHat.merge(forecast_Norway_KaggleMart_KaggleHat, on='date')
test_Norway_KaggleMart_KaggleSticker = test_Norway_KaggleMart_KaggleSticker.merge(forecast_Norway_KaggleMart_KaggleSticker, on='date')
test_Norway_KaggleRama_KaggleMug = test_Norway_KaggleRama_KaggleMug.merge(forecast_Norway_KaggleRama_KaggleMug, on='date')
test_Norway_KaggleRama_KaggleHat = test_Norway_KaggleRama_KaggleHat.merge(forecast_Norway_KaggleRama_KaggleHat, on='date')
test_Norway_KaggleRama_KaggleSticker = test_Norway_KaggleRama_KaggleSticker.merge(forecast_Norway_KaggleRama_KaggleSticker, on='date')
test_Sweden_KaggleMart_KaggleMug = test_Sweden_KaggleMart_KaggleMug.merge(forecast_Sweden_KaggleMart_KaggleMug, on='date')
test_Sweden_KaggleMart_KaggleHat = test_Sweden_KaggleMart_KaggleHat.merge(forecast_Sweden_KaggleMart_KaggleHat, on='date')
test_Sweden_KaggleMart_KaggleSticker = test_Sweden_KaggleMart_KaggleSticker.merge(forecast_Sweden_KaggleMart_KaggleSticker, on='date')
test_Sweden_KaggleRama_KaggleMug = test_Sweden_KaggleRama_KaggleMug.merge(forecast_Sweden_KaggleRama_KaggleMug, on='date')
test_Sweden_KaggleRama_KaggleHat = test_Sweden_KaggleRama_KaggleHat.merge(forecast_Sweden_KaggleRama_KaggleHat, on='date')
test_Sweden_KaggleRama_KaggleSticker = test_Sweden_KaggleRama_KaggleSticker.merge(forecast_Sweden_KaggleRama_KaggleSticker, on='date')

In [ ]:
frames = [test_Finland_KaggleMart_KaggleMug, test_Finland_KaggleMart_KaggleHat, test_Finland_KaggleMart_KaggleSticker,test_Finland_KaggleRama_KaggleMug,test_Finland_KaggleRama_KaggleHat,test_Finland_KaggleRama_KaggleSticker,test_Norway_KaggleMart_KaggleMug,test_Norway_KaggleMart_KaggleHat,test_Norway_KaggleMart_KaggleSticker,test_Norway_KaggleRama_KaggleMug,test_Norway_KaggleRama_KaggleHat,test_Norway_KaggleRama_KaggleSticker,test_Sweden_KaggleMart_KaggleMug,test_Sweden_KaggleMart_KaggleHat,test_Sweden_KaggleMart_KaggleSticker,test_Sweden_KaggleRama_KaggleMug,test_Sweden_KaggleRama_KaggleHat,test_Sweden_KaggleRama_KaggleSticker]
result = pd.concat(frames)

In [ ]:
result

In [ ]:
result['date'] =pd.to_datetime(result.date)
result = result.sort_values(by=['date'])

In [ ]:
result = result.drop(['date', 'country','store','product'], axis=1)

In [ ]:
sample_submission.head()

In [ ]:
result = result.sort_values(by=['row_id'])

In [ ]:
result.head()

In [ ]:
result.to_csv('submission.csv', index=False)
result.head()

Have any questions? Just ask me in the comment section